### Text Selection Data Merge

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Data/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [9]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [10]:
#def multi_wordgroup_in_wordgroup(word_list, df_target, target_column):
#    source_wordgroup_list = word_list
#    df_target2 = df_target
#    target_column2 = target_column
#
#    searchlist = []
#    resultlist = []
#    manager = multiprocessing.Manager()
#    resultlist = manager.list()
#    searchlist = manager.list()
#
#    def word_in_wordgroup(wordgroup):        
#        try:
#            word_in_word_cluster = df_target2[df_target2[f"{target_column2}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
#            for result in word_in_word_cluster[f"{target_column2}"]:
#                searchlist.append(wordgroup)
#                resultlist.append(result)
#        except:
#            pass
#
#    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
#        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste
#
#    search_list = list(searchlist)
#    result_list = list(resultlist)
#    df_result = pd.DataFrame([search_list,result_list]).transpose()
#    df_result.columns = ["word",f"{target_column}"]
#    df_result
#
#    return df_result

#### Text Data Read

In [11]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
44995,hayırbu,871
44996,konferansına,871
44997,kardeşisin,871
44998,kaldırıyorsun,871


In [12]:
#df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected.csv")
#df_twogram_select = df_twogram_select.head(1000)
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
213666,olmadan gerçek,52
213667,epey kolay,52
213668,hafızan iyi,52
213669,senin yaşındaydım,52


In [13]:
#df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected.csv")
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
307731,arkadaş mı olmak,38
307732,olan birkaç şey,38
307733,da size anlatayım,38
307734,için olay yerine,38


In [14]:
#df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected.csv")
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
388274,söz veriyorum kurul iş,16
388275,çıkmak için can atıyor,16
388276,ve içinden alevlerin çıktığı,16
388277,ne zaman isterse ama,16


In [15]:
#df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected.csv")
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
122369,ile matrix arasında seçim yapmam,19
122370,hey o iyi bir adam,19
122371,beklemek için nasıl bir yer,19
122372,yani bak normalde beni bilirsin,19


In [ ]:
#df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected.csv")
df_sixgram_select

In [ ]:
#df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected.csv")
df_sevengram_select

In [ ]:
#df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected.csv")
df_eightgram_select

In [ ]:
#df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected.csv")
df_ninegram_select

In [ ]:
#df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected.csv")
df_tengram_select

#### Text Data Word-Word Group In Word Group

In [16]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

In [17]:
#multi_wordgroup_in_wordgroup(word_list, df_twogram_select, "twogram")

##### Word

In [18]:
# 1 Way Slow
#df_word_in_two = word_in_wordgroup(word_list, df_twogram_select, "twogram", simple=True)
#df_word_in_two

In [19]:
# 2 Way Fast
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "twogram"
df_target = df_twogram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_two

,word,twogram
0,kazanıyorum,para kazanıyorum
1,kazanıyorum,böyle kazanıyorum
2,kazanıyorum,hayatımı kazanıyorum
3,kazanıyorum,dolar kazanıyorum
4,kazanıyorum,ben kazanıyorum
...,...,...
426664,ayrıntısına,ayrıntısına kadar
426665,ayrıntısına,ince ayrıntısına
426666,kardeşisin,onun kardeşisin
426667,kardeşisin,kız kardeşisin


In [20]:
df_word_in_two["word"].nunique()

29823

In [21]:
df_word_in_two_list = df_word_in_two.groupby(["word"])["twogram"].apply(list).reset_index(name="twogram")
df_word_in_two_list

,word,twogram
0,a,"[l a, a evet, a j, a b, a aziz, and a, a ve, t..."
1,aa,"[aa evet, şey aa, aa bu, aa şey, aa öyle, aa h..."
2,aaa,[aaa şey]
3,aah,"[aah o, aah en]"
4,aang,"[aang adında, aang adlı]"
...,...,...
29818,şık,"[çok şık, şık bir, şık görünüyorsun, kadar şık..."
29819,şımarık,"[şımarık bir, şımarık ve, seni şımarık]"
29820,şınav,[şınav çek]
29821,şırınga,[bir şırınga]


In [22]:
df_word_in_two_list.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

##### Twogram

In [23]:
# 1 Way Slow
#df_two_in_three = word_in_wordgroup(twogram_list, df_threegram_select, "threegram", simple=True)
#df_two_in_three.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_three

In [24]:
# 2 Way Fast
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_three = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_three.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_three

,twogram,threegram
0,korkutmak istemedim,seni korkutmak istemedim
1,korkutmak istemedim,sizi korkutmak istemedim
2,farkında değilsin,yaptığının farkında değilsin
3,farkında değilsin,olduğunun farkında değilsin
4,farkında değilsin,dediğinin farkında değilsin
...,...,...
557791,araba şirketi,araba şirketi için
557792,bir ederim,yerle bir ederim
557793,kaşık daha,bir kaşık daha
557794,olmadan gerçek,olmadan gerçek bir


In [25]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

,twogram,threegram
0,a artı,[a artı b]
1,a b,"[a b c, a b ve]"
2,a de,[l a de]
3,a e,[a e i]
4,a ile,"[a ile başlayan, a ile başlıyor]"
...,...,...
128721,şık bir,"[çok şık bir, şık bir yer, şık bir şey, şık bi..."
128722,şık değil,"[çok şık değil, şık değil mi]"
128723,şık görünüyorsun,[çok şık görünüyorsun]
128724,şık ve,[çok şık ve]


In [26]:
#df_two_in_three_list.to_excel("Two_In_Three_list.xlsx", index=False)

In [ ]:
df_two_in_four = word_in_wordgroup(twogram_list, df_fourgram_select, "fourgram", simple=True)
df_two_in_four.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_four

In [ ]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

In [ ]:
df_two_in_five = word_in_wordgroup(twogram_list, df_fivegram_select, "fivegram", simple=True)
df_two_in_five.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_five

In [ ]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

In [ ]:
df_two_in_six = word_in_wordgroup(twogram_list, df_sixgram_select, "sixgram", simple=True)
df_two_in_six.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_six

In [ ]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

In [ ]:
df_two_in_seven = word_in_wordgroup(twogram_list, df_sevengram_select, "sevengram", simple=True)
df_two_in_seven.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_seven

In [ ]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

In [ ]:
df_two_in_eight = word_in_wordgroup(twogram_list, df_eightgram_select, "eightgram", simple=True)
df_two_in_eight.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_eight

In [ ]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

In [ ]:
df_two_in_nine = word_in_wordgroup(twogram_list, df_ninegram_select, "ninegram", simple=True)
df_two_in_nine.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_nine

In [ ]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

In [ ]:
df_two_in_ten = word_in_wordgroup(twogram_list, df_tengram_select, "tengram", simple=True)
df_two_in_ten.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_ten

In [ ]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

In [ ]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

In [ ]:
df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)

##### Threegram

In [ ]:
df_three_in_four = word_in_wordgroup(threegram_list, df_fourgram_select, "fourgram", simple=True)
df_three_in_four.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_four

In [ ]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

In [ ]:
df_three_in_five = word_in_wordgroup(threegram_list, df_fivegram_select, "fivegram", simple=True)
df_three_in_five.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_five

In [ ]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

In [ ]:
df_three_in_six = word_in_wordgroup(threegram_list, df_sixgram_select, "sixgram", simple=True)
df_three_in_six.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_six

In [ ]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

In [ ]:
df_three_in_seven = word_in_wordgroup(threegram_list, df_sevengram_select, "sevengram", simple=True)
df_three_in_seven.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_seven

In [ ]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

In [ ]:
df_three_in_eight = word_in_wordgroup(threegram_list, df_eightgram_select, "eightgram", simple=True)
df_three_in_eight.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_eight

In [ ]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

In [ ]:
df_three_in_nine = word_in_wordgroup(threegram_list, df_ninegram_select, "ninegram", simple=True)
df_three_in_nine.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_nine

In [ ]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

In [ ]:
df_three_in_ten = word_in_wordgroup(threegram_list, df_tengram_select, "tengram", simple=True)
df_three_in_ten.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_ten

In [ ]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

In [ ]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [ ]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

In [ ]:
df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)

##### Fourgram

In [ ]:
df_four_in_five = word_in_wordgroup(fourgram_list, df_fivegram_select, "fivegram", simple=True)
df_four_in_five.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
df_four_in_six = word_in_wordgroup(fourgram_list, df_sixgram_select, "sixgram", simple=True)
df_four_in_six.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
df_four_in_seven = word_in_wordgroup(fourgram_list, df_sevengram_select, "sevengram", simple=True)
df_four_in_seven.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
df_four_in_eight = word_in_wordgroup(fourgram_list, df_eightgram_select, "eightgram", simple=True)
df_four_in_eight.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
df_four_in_nine = word_in_wordgroup(fourgram_list, df_ninegram_select, "ninegram", simple=True)
df_four_in_nine.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
df_four_in_ten = word_in_wordgroup(fourgram_list, df_tengram_select, "tengram", simple=True)
df_four_in_ten.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)

##### Fivegram

In [ ]:
df_five_in_six = word_in_wordgroup(fivegram_list, df_sixgram_select, "sixgram", simple=True)
df_five_in_six.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
df_five_in_seven = word_in_wordgroup(fivegram_list, df_sevengram_select, "sevengram", simple=True)
df_five_in_seven.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
df_five_in_eight = word_in_wordgroup(fivegram_list, df_eightgram_select, "eightgram", simple=True)
df_five_in_eight.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
df_five_in_nine = word_in_wordgroup(fivegram_list, df_ninegram_select, "ninegram", simple=True)
df_five_in_nine.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
df_five_in_ten = word_in_wordgroup(fivegram_list, df_tengram_select, "tengram", simple=True)
df_five_in_ten.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)

##### Sixgram

In [ ]:
df_six_in_seven = word_in_wordgroup(sixgram_list, df_sevengram_select, "sevengram", simple=True)
df_six_in_seven.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
df_six_in_eight = word_in_wordgroup(sixgram_list, df_eightgram_select, "eightgram", simple=True)
df_six_in_eight.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
df_six_in_nine = word_in_wordgroup(sixgram_list, df_ninegram_select, "ninegram", simple=True)
df_six_in_nine.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
df_six_in_ten = word_in_wordgroup(sixgram_list, df_tengram_select, "tengram", simple=True)
df_six_in_ten.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)

##### Sevengram

In [ ]:
df_seven_in_eight = word_in_wordgroup(sevengram_list, df_eightgram_select, "eightgram", simple=True)
df_seven_in_eight.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_eight

In [ ]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

In [ ]:
df_seven_in_nine = word_in_wordgroup(sevengram_list, df_ninegram_select, "ninegram", simple=True)
df_seven_in_nine.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_nine

In [ ]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

In [ ]:
df_seven_in_ten = word_in_wordgroup(sevengram_list, df_tengram_select, "tengram", simple=True)
df_seven_in_ten.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_ten

In [ ]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

In [ ]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [ ]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

In [ ]:
df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)

##### Eightgram

In [ ]:
df_eight_in_nine = word_in_wordgroup(eightgram_list, df_ninegram_select, "ninegram", simple=True)
df_eight_in_nine.rename(columns={"word":"eightgram"}, inplace=True)
df_eight_in_nine

In [ ]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

In [ ]:
df_eight_in_ten = word_in_wordgroup(eightgram_list, df_tengram_select, "tengram", simple=True)
df_eight_in_ten.rename(columns={"word":"eightgram"}, inplace=True)
df_eight_in_ten

In [ ]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

In [ ]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [ ]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

In [ ]:
df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)

##### Ninegram

In [ ]:
df_nine_in_ten = word_in_wordgroup(ninegram_list, df_tengram_select, "tengram", simple=True)
df_nine_in_ten.rename(columns={"word":"ninegram"}, inplace=True)
df_nine_in_ten

In [ ]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

In [ ]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [ ]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

In [ ]:
df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)

##### Text Data Word-Word Group In Word Group Merge

In [ ]:
df_1_2_3_merge = pd.merge(df_word_in_two, df_two_in_three, how="left", on="twogram")
df_1_2_3_merge.drop_duplicates(inplace=True)
df_1_2_3_merge

In [ ]:
df_1_2_3_4_merge = pd.merge(df_1_2_3_merge, df_three_in_four, how="left", on="threegram")
df_1_2_3_4_merge.drop_duplicates(inplace=True)
df_1_2_3_4_merge

In [ ]:
df_1_2_3_4_5_merge = pd.merge(df_1_2_3_4_merge, df_four_in_five, how="left", on="fourgram")
df_1_2_3_4_5_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_merge

In [ ]:
df_1_2_3_4_5_6_merge = pd.merge(df_1_2_3_4_5_merge, df_five_in_six, how="left", on="fivegram")
df_1_2_3_4_5_6_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_merge

In [ ]:
df_1_2_3_4_5_6_7_merge = pd.merge(df_1_2_3_4_5_6_merge, df_six_in_seven, how="left", on="sixgram")
df_1_2_3_4_5_6_7_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_merge

In [ ]:
df_1_2_3_4_5_6_7_8_merge = pd.merge(df_1_2_3_4_5_6_7_merge, df_seven_in_eight, how="left", on="sevengram")
df_1_2_3_4_5_6_7_8_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_merge = pd.merge(df_1_2_3_4_5_6_7_8_merge, df_eight_in_nine, how="left", on="eightgram")
df_1_2_3_4_5_6_7_8_9_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_9_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge = pd.merge(df_1_2_3_4_5_6_7_8_9_merge, df_nine_in_ten, how="left", on="ninegram")
df_1_2_3_4_5_6_7_8_9_10_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_9_10_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge["word"].nunique()

In [ ]:
word_count_result(df_1_2_3_4_5_6_7_8_9_10_merge,["twogram","threegram","fourgram","fivegram","sixgram","sevengram","eightgram","ninegram","tengram"])

In [ ]:
#df_1_2_3_4_5_6_7_8_9_10_merge.to_excel(f"{lang_folder.capitalize()}_Text_Selection_Data.xlsx", index=False)

In [ ]:
#df_1_2_3_4_5_6_7_8_9_10_freq_merge = pd.merge(df_1_2_3_4_5_6_7_8_9_10_merge, df_word_select, how="inner", on="word")
df_1_2_3_4_5_6_7_8_9_10_freq_merge = pd.merge(df_word_select, df_1_2_3_4_5_6_7_8_9_10_merge, how="left", on="word")
df_1_2_3_4_5_6_7_8_9_10_freq_merge.drop_duplicates(inplace=True)
#df_1_2_3_4_5_6_7_8_9_10_freq_merge.sort_values(by="frequency", inplace=True, ascending=False)
df_1_2_3_4_5_6_7_8_9_10_freq_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["word"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["tengram"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["ninegram"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["eightgram"].nunique()

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result.xlsx")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_text_file = pd.read_excel("Turkish_Text_Selection_Data.xlsx")
df_text_file

In [ ]:
df_text_file["word"].nunique()

In [ ]:
df_text_file.groupby("word")[["twogram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("twogram")[["threegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("threegram")[["fourgram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("fourgram")[["fivegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("fivegram")[["sixgram"]].agg(lambda x : set(x))

In [ ]:
df_test = df_text_file.groupby("fivegram")[["sixgram"]].agg(lambda x : set(x))
df_test.reset_index(inplace=True)
df_test

In [ ]:
fivegram_list = df_test["fivegram"].to_list()

In [ ]:
df_fivegram_select[~df_fivegram_select["fivegram"].isin(fivegram_list)]

In [ ]:
df_twogram_select[df_twogram_select["twogram"]=="söz vermedim"]

In [ ]:
df_threegram_select[df_threegram_select["threegram"]=="öyle bir söz"]

In [ ]:
df_text_file.groupby("sixgram")[["sevengram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("sevengram")[["eightgram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("eightgram")[["ninegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("ninegram")[["tengram"]].agg(lambda x : set(x))

In [ ]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram_select, df_fourgram_select, df_fivegram_select], axis=1)
df_ngram_concat

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup_simple(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup_simple(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [ ]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [ ]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [ ]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

In [ ]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="right", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

In [ ]:
df_ngram_sample_join_merge.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [ ]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

In [ ]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

In [ ]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

In [ ]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

In [ ]:
df_ngram_sample_concat.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [ ]:
df_ngram_threegram_unique = df_ngram_sample_concat[["threegram","three_freq"]].drop_duplicates()
df_ngram_fourgram_unique = df_ngram_sample_concat[["fourgram","four_freq"]].drop_duplicates()
df_ngram_fivegram_unique = df_ngram_sample_concat[["fivegram","five_freq"]].drop_duplicates()

In [ ]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_threegram_unique["three_freq"].sum()
fourgram_result_freq = df_ngram_fourgram_unique["four_freq"].sum()
fivegram_result_freq = df_ngram_fivegram_unique["five_freq"].sum()

In [ ]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [ ]:
(twogram_result_freq/twogram_select_freq)*100

In [ ]:
(threegram_result_freq/threegram_select_freq)*100

In [ ]:
(fourgram_result_freq/fourgram_select_freq)*100

In [ ]:
(fivegram_result_freq/fivegram_select_freq)*100

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 4

In [ ]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
word_list = df_word_select["word"].to_list()

In [ ]:
df_two_three = pd.read_excel("/home/kurubal/Downloads/Twogram Threegram.xlsx")
df_two_three

In [ ]:
word_count_result(df_two_three,["threegram"])

In [ ]:
df_two_three["twogram"].count()

In [ ]:
df_two_three["threegram"].count()

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_two_three, "twogram","threegram",threegram_sample)
df_two_in_three

In [ ]:
set_all_twogram = set(df_two_three["twogram"].dropna().to_list())
set_twogram_in_threegram = set(df_two_in_three["twogram"].to_list())

In [ ]:
set_all_twogram.difference(set_twogram_in_threegram)

In [ ]:
set_threegram_cover_twogram = set(df_two_in_three["threegram"].to_list())
df_selected_threegram = pd.DataFrame(set_threegram_cover_twogram,columns=["threegram"])
df_selected_threegram

In [ ]:
df_selected_threegram.to_excel("Threegram_Selected.xlsx", index=False)

In [ ]:
word_count_result(df_selected_threegram,["threegram"])

In [ ]:
df_threegram_word_usage_select = word_usage_result(word_list, df_selected_threegram, "threegram", word_use_num_min, word_use_num_max)
df_threegram_word_usage_select

In [ ]:
word_count_result(df_threegram_word_usage_select,["threegram"])